In [3]:
#Visualizations For Dashboard


import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from icecream import ic
import matplotlib.pyplot as plt 
import plotly.io as pio
import datetime as dt
from scipy import stats



In [4]:
#loading the data
pio.templates.default = "plotly_white"
ic.disable()
azure_data = pd.read_csv('/media/seqcuredata/dailyjobs/New_Dashboard_Dataframe_AZURE.csv') 
aws_data = pd.read_csv('/media/seqcuredata/dailyjobs/New_Dashboard_Dataframe_AWS.csv')
azure_data_fail = pd.read_csv('/media/seqcuredata/dailyjobs/FAILED_Dashboard_Dataframe_AZURE.csv')
aws_data_fail = pd.read_csv('/media/seqcuredata/dailyjobs/FAILED_Dashboard_Dataframe_AWS.csv')
azure_data_cost = pd.read_csv('/media/seqcuredata/dailyjobs/COST_Dashboard_Dataframe_AZURE.csv')
aws_data_cost = pd.read_csv('/media/seqcuredata/dailyjobs/COST_Dashboard_Dataframe_AWS.csv')
for index, row in azure_data_cost.iterrows():
    azure_data_cost = azure_data_cost[azure_data_cost['Job Status'] == 'JobStatus.DONE']

for index, row in aws_data_cost.iterrows():
    aws_data_cost = aws_data_cost[aws_data_cost['Job Status'] == 'JobStatus.DONE']


status_azure = pd.read_csv('/media/seqcuredata/dailyjobs/AZURE_Target_Status.csv')
status_aws = pd.read_csv('/media/seqcuredata/dailyjobs/AWS_Target_Status.csv')

removed_column = azure_data.columns[0]
azure_data = azure_data.drop(columns=[removed_column])

removed_column2 = aws_data.columns[0]
aws_data = aws_data.drop(columns=[removed_column2])



In [3]:
"""removed_column_cost = azure_data_cost.columns[0]
azure_data_cost = azure_data_cost.drop(columns=[removed_column_cost])


removed_column2_cost = aws_data_cost.columns[0]
aws_data_cost = aws_data_cost.drop(columns=[removed_column2_cost])"""
        # retrieve fidelities from aws and azure data csv
        # add them to the dmerge the fidelities from the aws data and azure data together 
        # merge the cost estimates from 
        # add the fidelity from the aws data and azure data dataframe into 

"""azure_fidelity = azure_data["Fidelity"] 
aws_fidelity = aws_data["Fidelity"]
fidelities = pd.concat([azure_fidelity, aws_fidelity], axis=1)
print(fidelities)"""
# merge this all into one series 

# 

azure_merge = azure_data_cost [["Target", "Number of Qubits", "Fidelity", "Cost Estimate"]]

aws_merge = aws_data_cost [["Target", "Number of Qubits", "Fidelity", "Cost Estimate"]]


dataframe = [azure_merge, aws_merge]

both_provider_data = pd.concat(dataframe)
#both_provider_all = pd.concat([both_provider_data, fidelities], axis=0, ignore_index = True)

both_provider_data["Number of Qubits"] = both_provider_data["Number of Qubits"].astype(str)

both_filtered_scatter = both_provider_data[both_provider_data['Number of Qubits'].str.contains('8')]
both_filtered_scatter['size'] = (both_filtered_scatter['Cost Estimate'] - both_filtered_scatter['Cost Estimate'].min()) / (both_filtered_scatter['Cost Estimate'].max() - both_filtered_scatter['Cost Estimate'].min()) * 50 + 10
fig = px.scatter(both_filtered_scatter, x = 'Cost Estimate', y = 'Fidelity', color = 'Target', size = 'size')
        #fig.update_traces(marker=dict(size=30))
fig.update_xaxes(type='log', title = "Cost Total [in log scale]")
fig 

/tmp/ipykernel_998077/161336881.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_filtered_scatter['size'] = (both_filtered_scatter['Cost Estimate'] - both_filtered_scatter['Cost Estimate'].min()) / (both_filtered_scatter['Cost Estimate'].max() - both_filtered_scatter['Cost Estimate'].min()) * 50 + 10


In [4]:
azure_fidelity = azure_data["Fidelity"] 
aws_fidelity = aws_data["Fidelity"]
fidelities = pd.concat([azure_fidelity, aws_fidelity], axis=1)
print(fidelities)



     Fidelity  Fidelity
0    0.130062  0.144723
1    0.626334  0.671745
2    0.675497  0.009863
3    0.015363  0.107876
4    0.125071  0.141224
..        ...       ...
272  0.121477       NaN
273  0.139794       NaN
274  0.600362       NaN
275  0.662499       NaN
276  0.647610       NaN

[277 rows x 2 columns]


In [5]:
frames = [status_azure, status_aws]
both_provider_status = pd.concat(frames)

both_provider_status = both_provider_status[["Target", "Target Status"]]
print(both_provider_status)

                Target Target Status
0      ionq.qpu.aria-1   Unavailable
1      ionq.qpu.aria-2     Available
2             ionq.qpu     Available
3       ionq.simulator     Available
4  quantinuum.qpu.h1-1      Degraded
5  quantinuum.qpu.h2-1      Degraded
6  rigetti.qpu.ankaa-2     Available
0               Aria 1       OFFLINE
1               Aria 2        ONLINE
2            Aspen-M-3       OFFLINE
3               Garnet        ONLINE
4              Harmony        ONLINE


In [6]:
#qpu , qpu status, provider 
azure_data_fail["Provider"] = "Azure"
#cut out stuff BEFORE
azure_data_fail["Target Status"] = azure_data_fail["Target Status"].astype(str)
azure_data_fail['Target Status'] = azure_data_fail['Target Status'].str.split("index Target Status\n0       0 ").str[1]

aws_data_fail["Provider"] = "AWS"
azure_tomerge_status = azure_data_fail[["Target Status", "Provider", "Target", "Job ID"]]


aws_tomerge_status = aws_data_fail[["Target Status", "Provider", "Target", "Job ID"]]


frames = [azure_tomerge_status, aws_tomerge_status]
both_provider_data_table = pd.concat(frames) #this is big
grouped_qpu= both_provider_data_table.groupby("Target")["Job ID"].count().reset_index().rename(mapper = {"Job ID":"num_total_jobs"}, axis  = 1)
error_df = azure_data_fail.groupby(["Target", "Target Status"])["Job ID"].count().reset_index().rename(mapper = {"Job ID":"error_count"}, axis  = 1)

#rouped_qpu = both_provider_data_table.groupby("Target")

merged_df = error_df.merge(grouped_qpu)
#cut out stuff BEFORE

print(merged_df)
        

KeyError: 'Target Status'

In [ ]:
# COST ESTIMATE AWS
fig = px.bar(aws_data_cost, x = 'Target', y = 'Cost Estimate', color = 'Target', color_discrete_sequence=["#4dac26","#92c5de" , "#dfc27d"])
fig.update_yaxes(type='linear', title = "Cost Estimate [in linear scale]")
fig.update_layout(showlegend=False)

fig


In [ ]:
# COST ESTIMATE AZURE 
# 1) i need to group the cost estimates by target
# 2) x axis shows the targets, y axis the cost estimate 
fig = px.box(azure_data_cost, x = 'Target', y = 'Cost Estimate', title = 'Cost Estimates (USD) vs. QPU',  color = 'Target')
fig.update_yaxes( title = "Cost Estimate (in log scale)")

#fig.update_xaxes(color = ["sandybrown","seagreen","tomato","violet","wheat","steelblue"])

#fig.update_yaxes(log_y = True)
#azure_data_cost['Cost Estimate'].plot(kind='hist', logy=True)
fig 


#cost_estimate = azure_data_cost['Cost Estimate']
#print(cost_estimate)

In [7]:
#final thing 
#group by the target 
#need to find count of jobs ran, then count of failed jobs, then count of each specific value from error message


#cut out stuff AFTER orthogonal bc it is diff for each job(too many colors were showing up for the same error)
azure_data_fail['Failure Message'] = azure_data_fail['Failure Message'].str.split('orthogonal').str[0]
#cut out stuff BEFORE
azure_data_fail['Failure Message'] = azure_data_fail['Failure Message'].str.split("'code':").str[1]



count_df = azure_data_fail.groupby("Target")["Job ID"].count().reset_index().rename(mapper = {"Job ID":"num_total_jobs"}, axis  = 1)
 

#counts of total failures
 
failure_df = azure_data_fail.groupby("Target")["Job Status"].apply(lambda x: (x == "JobStatus.ERROR").sum()).reset_index().rename(mapper = {"Job Status":"num_failed_jobs"}, axis  = 1)
 
 
# counts of failures by type
 
error_df = azure_data_fail.groupby(["Target", "Failure Message"])["Job ID"].count().reset_index().rename(mapper = {"Job ID":"error_count"}, axis  = 1)


In [8]:
# merge df
 
merged_df = error_df.merge(count_df).merge(failure_df)
 
#get proportions
 
merged_df["failure_rate"] = (merged_df["num_failed_jobs"] / merged_df["num_total_jobs"])*100

#use this  for y 
merged_df["error_rate"] = merged_df["failure_rate"] * (merged_df["error_count"] / merged_df["num_failed_jobs"])

merged_df["Failure Message"] = merged_df["Failure Message"].astype(str)
substring = 'NotEnoughQuota'
filter = merged_df['Failure Message'].str.contains(substring)
merged_df = merged_df[~filter]

merged_df

,Target,Failure Message,error_count,num_total_jobs,num_failed_jobs,failure_rate,error_rate
0,quantinuum.qpu.h1-1,"'400', 'message': '81 - Duplicate job id'}",1,69,2,2.898551,1.449275
2,quantinuum.qpu.h2-1,"'400', 'message': '81 - Duplicate job id'}",2,88,3,3.409091,2.272727
4,rigetti.qpu.ankaa-2,"'InvalidInputData', 'message': 'Problem compi...",51,88,52,59.090909,57.954545
5,rigetti.qpu.ankaa-2,"'TransientFailure', 'message': 'Internal error'}",1,88,52,59.090909,1.136364


In [9]:
#plotting 
merged_df.loc[merged_df['Failure Message'].str.contains("'400', 'message': '81 - Duplicate job id'", na=False), 'Failure Message'] = 'Duplicate Job ID'
merged_df.loc[merged_df['Failure Message'].str.contains("'InvalidInputData'", na=False), 'Failure Message'] = 'Invalid Input Data'
merged_df.loc[merged_df['Failure Message'].str.contains("'TransientFailure'", na=False), 'Failure Message'] = 'Transient Failure'


fig = px.bar(merged_df, x= "Target", y="error_rate", color = "Failure Message", barmode= "stack",
             title = "Failure Rate VS Target")
fig.update_layout(showlegend =True, yaxis_title = "Failure Rate (%)", xaxis_title = "QPUs With Failed Runs", legend_title_text = "Error") 

merged_df["Failure Message"] = merged_df['Failure Message'].astype(str)
#cutting out the message after
legend_labels = {subcategory: subcategory.split(", '")[0] for subcategory in merged_df['Failure Message'].unique()}
 
# Update the legend 
for trace in fig.data:
    trace.name = legend_labels.get(trace.name, trace.name)

fig

In [ ]:
# azure plot 1 
# how does fidelity change over time- line chart. x axis is time y is fidelity, color by the machine,
#  make interactive so facet by number qubits

In [ ]:
#home page 

#AZURE 

azure_data["Number of Qubits"] = azure_data["Number of Qubits"].astype(str)
fig= px.line(azure_data, x = "End Time", y ="Fidelity", facet_col = "Number of Qubits",
             color ="Target",category_orders={"Number of Qubits": ["8","10","12"]}
 )
fig.update_layout(showlegend= False)

In [7]:
#combine the 2 df. make a column for provider and put aws or azure. then plot as normal
#SIMILAR;  job id, targte, number of qubits, end time, fidelity 
azure_data["Provider"] = "Azure"

aws_data["Provider"] = "AWS"

azure_tomerge = azure_data [["Job ID", "Target", "Number of Qubits", "Fidelity", "End Time", "Provider"]]

aws_tomerge = aws_data[["Job ID", "Target", "Number of Qubits", "Fidelity", "End Time","Provider"]]


frames = [azure_tomerge, aws_tomerge]
both_provider_data = pd.concat(frames)
print(both_provider_data["Target"].unique())
both_provider_data["Number of Qubits"] = both_provider_data["Number of Qubits"].astype(str)
fig= px.line(both_provider_data, x = "End Time", y ="Fidelity",
              title = "Fidelity VS End Time",
             color = "Target",
              facet_col= "Number of Qubits")

fig

['quantinuum.sim.h1-1e' 'Garnet']


In [ ]:
fig = px.line(both_provider_data, x = "End Time", y ="Fidelity",
              title = "Fidelity VS End Time",
             color = "Provider",
              facet_col= "Number of Qubits")

fig

In [ ]:
#AWS 
#making the scale discrete bc it was continuous before 
aws_data["Number of Qubits"] = aws_data["Number of Qubits"].astype(str)
fig= px.line(aws_data, x = "End Time", y ="Fidelity",
              title = "Fidelity VS End Time",
             color = "Number of Qubits",
              facet_col= "Number of Qubits")

fig.update_layout(showlegend= False)


In [ ]:
#AVG QUEUE TIME VS TARGET
azure_data["Average Queue Time"]= azure_data["Average Queue Time"].div(3600)
fig = px.box(azure_data , x= "Target", y = "Average Queue Time",
 title = "Average Queue Time VS Target", color = 'Target'

# need to figure out how to change the color of the label        
)
#fig.update_layout("Ionq simulator has no average queue time because it is not running on real quantum hardware.")
fig  
#convert from sec to hours then use , only azure , remove legend, 
# rigetti under a min ,ionq 15 days, change y axis to log scale - tick for 1 min/hour

In [ ]:
# RUN TIME GRAPH AZURE 


fig = px.box(azure_data, x= "Target", y = "Run Time",
 title = "Run Time (Queue Time + Execution Time) VS Target", color = 'Target',

# need to figure out how to change the color of the label        
)
fig.update_yaxes(title = "Run Time [hours]")
# add a unit to the y axis to specify that it is hours 
fig 
#fig.update_layout("Ionq simulator has no average queue time because it is not running on real quantum hardware.")

In [ ]:
# RUN TIME FOR AWS DATA
fig = px.box(aws_data , x= "Target", y = "Run Time",
 title = "Run Time (Queue Time + Execution Time) VS Target", color = 'Target'

# need to figure out how to change the color of the label        
)
fig.update_yaxes(title = "Run Time [hours]")
#fig.update_layout("Ionq simulator has no average queue time because it is not running on real quantum hardware.")
fig 

In [ ]:
# AVERAGE QUEUE TIME VS HOW LONG IN QUEUE (ACTUAL TIME) FOR AZURE
fig = px.scatter(azure_data, x = "Time in Queue", y = "Average Queue Time", title = "Average Queue Time (reported) VS Actual Queue Time")
fig 
"""data = sns.heatmap(azure_data, annot=True)
heat = go.Heatmap(data,
    x = "How Long in Queue",
    y = "Average Queue Time",
    zmin = - 0.25, # Sets the lower bound of the color domain
    zmax = 1,
    xgap = 1, # Sets the horizontal gap (in pixels) between bricks
    ygap = 1,
    colorscale = 'RdBu'
)"""
#fig.update_xaxes(title = "Actual Queue Time [hours]")
# FOUND R VALUE
#r = stats.pearsonr(azure_data['How Long in Queue'], azure_data['Average Queue Time'])
"""azure_data["Average Queue Time"]= azure_data["Average Queue Time"].div(3600)
for index, row in azure_data.iterrows():
    queue_time_hour_actual = row["How Long in Queue"]
    queue_time_hour_avg = row["Average Queue Time"]

    r2 = r2_score(np.array(queue_time_hour_actual), np.array(queue_time_hour_avg)) 
            
    print(r2)"""

In [ ]:
#ic(aws_data)

In [ ]:
#azure_data.plot(x="Number of Qubits", y="Fidelity", kind="scatter",title="Fidelity vs. Qubits")

fig= px.box(azure_data, x = "Number of Qubits", y ="Fidelity", title = "Fidelity Rate VS Number of Qubits (Azure)",
            color  = "Number of Qubits",
              category_orders={"Number of Qubits": [8,10,12]}
              )
#titles 
fig
#as you add more gates/qubits/more noise, 
#picture of dashboard, put plots in showing options/drop down 
#finding an.


In [ ]:
#aws_data.plot(x="Number of Qubits", y="Fidelity", kind="scatter",title="Fidelity vs. Qubits")

#change to boxplot 




fig= px.box(aws_data, x = "Number of Qubits", y ="Fidelity", title = "Fidelity Rate VS Number of Qubits (AWS)",
            color  = "Number of Qubits",
              category_orders={"Number of Qubits": [8,10,12]}
              )
#titles 
fig.update_yaxes(title = "Fidelity Rate")

In [ ]:
#azure_data.plot(x="End Time", y="Fidelity", kind="scatter",title="Azure Fidelity vs. Time").tick_params(labelbottom=False)

In [ ]:
#aws_data.plot(x="End Time", y="Fidelity", kind="scatter",title="AWS Fidelity vs. Time").tick_params(labelbottom=False)